# Flipkart EDA assignment

## TODO: Upload csv file before moving to next

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Flipkart Data Engineering").getOrCreate()
file_path = '/FileStore/tables/Flipkart.csv' # it should path from dbfs or the full url
flipkart_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [0]:
flipkart_df.printSchema()
flipkart_df.count()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- maincateg: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- actprice1: integer (nullable = true)
 |-- norating1: integer (nullable = true)
 |-- noreviews1: integer (nullable = true)
 |-- star_5f: integer (nullable = true)
 |-- star_4f: integer (nullable = true)
 |-- star_3f: integer (nullable = true)
 |-- star_2f: integer (nullable = true)
 |-- star_1f: integer (nullable = true)
 |-- fulfilled1: integer (nullable = true)

Out[4]: 5244

In [0]:
flipkart_df.show(5)

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|      499|     3015|       404|   1458|    657|    397|    182|    321|         1|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|      999|      449|        52|    229|     70|     71|     33|     46|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|12593|Loafers For Men  ...|   3.9|      

In [0]:
# Display all category names

flipkart_df.select("maincateg").distinct().show()

+---------+
|maincateg|
+---------+
|     null|
|      Men|
|    Women|
+---------+



In [0]:
# Filter products with rating > 4.5 and more than 1000 reviews

flipkart_df.filter((flipkart_df["Rating"]>4.0) & (flipkart_df["noreviews1"]>100)).show()

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|13859|Women Black Wedge...|   4.2|    Women|Flipkart|      279|     3048|       487|   1746|    679|    348|    114|    161|         1|
|19453|Denill Ankle Leng...|   4.1|    Women|Flipkart|      999|     6806|       961|   3646|   1508|    810|    335|    510|         1|
| 3205|     SFG-23 Slippers|   4.1|      Men|Flipkart|      319|    12936|      1598|   7030|   2850|   1453|    587|   1016|         1|
| 7625|White casual, gym...|   4.2|      Men|Flipkart|      999|     2430|       381|   1472|    397|    251|    105|    205|         0|
|17791|Synthetic Leather...|   4.2|    Wo

In [0]:
# Display Products in 'Men' category that are fulfilled

flipkart_df.filter((flipkart_df["maincateg"]=="Men") & (flipkart_df["fulfilled1"]==1)).show()

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|10680|Combo Pack Of 2 C...|   3.9|      Men|Flipkart|     2400|      824|       105|    386|    199|    106|     61|     72|         1|
|11579|MOCCASSIN Slip On...|   3.8|      Men|Flipkart|     2299|     3066|       392|   1304|    803|    462|    189|    308|         1|
|12260|Long OfficeWear C...|   3.9|      

In [0]:
# Dsiplay number of products per category
from pyspark.sql.functions import count
flipkart_df.groupBy("maincateg").agg(count("title")).show()

+---------+------------+
|maincateg|count(title)|
+---------+------------+
|     null|          67|
|      Men|        2145|
|    Women|        3032|
+---------+------------+



In [0]:
# Display Average rating per category
from pyspark.sql.functions import avg
flipkart_df.groupBy("maincateg").agg(avg("Rating")).show()


+---------+------------------+
|maincateg|       avg(Rating)|
+---------+------------------+
|     null|             4.225|
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [0]:
# Dsiplay Category with highest average number of reviews
from pyspark.sql.functions import avg
flipkart_df.groupBy("maincateg").agg(avg("noreviews1").alias("avg of reviews")).orderBy("avg of reviews",ascending=False).show(1)


+---------+-----------------+
|maincateg|   avg of reviews|
+---------+-----------------+
|      Men|759.6554778554779|
+---------+-----------------+
only showing top 1 row



In [0]:
# Top 5 products with highest price. display product name and price

flipkart_df.select("title","actprice1").orderBy("actprice1",ascending=False).show(5)

+--------------------+---------+
|               title|actprice1|
+--------------------+---------+
|GEL-QUANTUM 360 K...|    15999|
|Women Orange Spor...|    12999|
|Hybrid Astro Runn...|    10999|
|GT-2000 5 (D) Run...|    10999|
|DEERUPT RUNNER W ...|     9999|
+--------------------+---------+
only showing top 5 rows



In [0]:
# Display Min, max, and avg price per category
from pyspark.sql.functions import min, max, avg

flipkart_df.groupBy("maincateg").agg(
    min("actprice1").alias("Min Price"),
    max("actprice1").alias("Max Price"),
    avg("actprice1").alias("Average Price")
).show()


+---------+---------+---------+------------------+
|maincateg|Min Price|Max Price|     Average Price|
+---------+---------+---------+------------------+
|     null|      499|     5999|1574.7313432835822|
|      Men|      139|    10999|  1455.86013986014|
|    Women|      162|    15999|1319.7081134564644|
+---------+---------+---------+------------------+



In [0]:
# Display number of nulls in each column

from pyspark.sql.functions import col, sum, when

# Count nulls for each column
null_counts = flipkart_df.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in flipkart_df.columns
])

null_counts.show()


+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| id|title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|  0|    0|   203|       67|       0|        0|        0|         0|     68|      0|      0|      0|    186|         0|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+



In [0]:
#calaculate and display the category name ,number of fullfilled, number of unfulilled
from pyspark.sql.functions import count, when

flipkart_df.groupBy("maincateg").agg(
    
    count(when(flipkart_df["fulfilled1"] == 1, True)).alias("Fulfilled Count"),
    count(when(flipkart_df["fulfilled1"] == 0, True)).alias("Unfulfilled Count")
).show()


+---------+---------------+-----------------+
|maincateg|Fulfilled Count|Unfulfilled Count|
+---------+---------------+-----------------+
|     null|             40|               27|
|      Men|           1352|              793|
|    Women|           1778|             1254|
+---------+---------------+-----------------+



In [0]:
# Display Count of products per category

flipkart_df.groupBy("maincateg").agg(count("title")).show()

+---------+------------+
|maincateg|count(title)|
+---------+------------+
|     null|          67|
|      Men|        2145|
|    Women|        3032|
+---------+------------+



In [0]:
# Display Average rating per category
from pyspark.sql.functions import avg
flipkart_df.groupBy("maincateg").agg(avg("Rating")).show()


+---------+------------------+
|maincateg|       avg(Rating)|
+---------+------------------+
|     null|             4.225|
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [0]:
# Dsiplay Category with highest average number of reviews
from pyspark.sql.functions import avg
flipkart_df.groupBy("maincateg").agg(avg("noreviews1").alias("avg of reviews")).orderBy("avg of reviews",ascending=False).show(1)


+---------+-----------------+
|maincateg|   avg of reviews|
+---------+-----------------+
|      Men|759.6554778554779|
+---------+-----------------+
only showing top 1 row



In [0]:
# Display Bar chart of product count per category

# Use Databricks UI to visualize this as a bar chart
newdf=flipkart_df.dropna()
display(newdf.groupBy("maincateg").agg(count("title")))


maincateg,count(title)
Men,2062
Women,2932


Databricks visualization. Run in Databricks to view.

In [0]:
# Bar chart of average rating per category
from pyspark.sql.functions import avg
newdf=flipkart_df.dropna()
display(newdf.groupBy("maincateg").agg(avg("Rating")))

maincateg,avg(Rating)
Men,3.967410281280314
Women,4.041507503410653


Databricks visualization. Run in Databricks to view.

In [0]:
# Display Bar chart of total number of reviews per category
from pyspark.sql.functions import sum
newdf=flipkart_df.dropna()
display(newdf.groupBy("maincateg").agg(sum("noreviews1")))

maincateg,sum(noreviews1)
Men,1543836
Women,505578


Databricks visualization. Run in Databricks to view.

In [0]:
# Display product name and 5 star rating for those products which have highest 5 star rating

from pyspark.sql.functions import max

flipkart_df.select("title", "star_5f").groupBy("title").agg(max("star_5f").alias("max_star_5f")).orderBy("max_star_5f", ascending=False).show()


+--------------------+-----------+
|               title|max_star_5f|
+--------------------+-----------+
|Jio-13 Running Sh...|     151193|
|Running Shoes For...|     151193|
|Jio-13 Running Sh...|     151159|
|VERTIGO BLK RUNNI...|      66369|
|Cosko Sports Shoe...|      64987|
|    Men Olive Sandal|      61722|
|Men Brown, Beige ...|      61722|
|Oxygen Running Sh...|      55231|
|Boxer-303 Sneaker...|      54272|
|Mesh,Cricket, Bad...|      45128|
|Mesh,Cricket, Bad...|      45128|
|Mesh,Cricket, Bad...|      45128|
|Mesh,Cricket, Bad...|      45127|
|SOCKS Running Sho...|      43374|
|Running Shoes For...|      42503|
|Combo Pack of 2 C...|      35264|
|  SFG-204 Flip Flops|      35161|
|Climber Boots For...|      33445|
|Climber Boots For...|      33443|
|Casuals For Men  ...|      32724|
+--------------------+-----------+
only showing top 20 rows

